In [ ]:
from numpy import asarray
from PIL import Image
from numpy import *
import numpy as np
import glob
import contextlib
import matplotlib.pyplot as plt

# Entrada da Imagem
imagem = input("Digite o nome do arquivo:\n")
image = Image.open("%s" %imagem)
im = asarray(Image.open("%s" %imagem))
imoriginal = Image.fromarray(im)  #salvamos a imagem original
h = im.shape[0] #Dimensão da altura
w = im.shape[1] #Dimensão da largura

if w!=h:
  size = max(image.size)
  image = image.resize((size, size))
  w, h = image.size

im = asarray(image)
imoriginal = Image.fromarray(im)
# Criamos valoresd de x e y, que vão mapear os pixels da imagem
x,y = meshgrid(range(w),range(h))
#xmap,ymap são a aplicação da transformação do gato de Arnold
#na qual, M(x, y) −→ T(x + y, x + 2y) mod h para pegar o resto pela dimensão da altura
# e mod w para pegar o resto pela dimensão da largura
xmap = (x+y) % w
ymap = (x+2*y) % h
i=1

print("O que deseja fazer?\n")
res = int(input("1 - Fazer um GIF da imagem passando pela transformação do Gato de Arnold.\n2 - Descriptografar uma imagem.\n3 - Criptografar uma imagem.\n"))
if res == 1:
  while(1):

    result = Image.fromarray(im)
    result.save("cat_%04d.png" % i)
    #condição de parada, quando o mapa de pixel se torna igual a imagem inicial
    if (result == imoriginal) & (i!=1):
        print("Rodou com %d iterações" % i);
        break;
    im = im[xmap,ymap]
    i=i+1

  # filepaths
  fp_in = "cat_*.png"
  fp_out = "a.gif"

  # use exit stack to automatically close opened images
  with contextlib.ExitStack() as stack:

      # lazily load images
      imgs = (stack.enter_context(Image.open(f))
              for f in sorted(glob.glob(fp_in)))

      # extract  first image from iterator
      img = next(imgs)

      # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
      img.save(fp=fp_out, format='GIF', append_images=imgs,
              save_all=True, duration=150, loop=0)


elif res == 2:
  p = int(input("Digite a iteração que a imagem está:\n"))
  while(1):

    result = Image.fromarray(im)
    #condição de parada, quando o mapa de pixel se torna igual a imagem inicial
    if (result == imoriginal) & (i!=1):
        print("%d" %i)
        break;
    im = im[xmap,ymap]
    i=i+1
  im = asarray(Image.open("%s" %imagem))
  for j in range(i-p+1):
      result = Image.fromarray(im)
      if (j == i-p-1):
      #Subtraímos por 1, pois ele começara a contagem a partir da iteração da frente, logo subtraímos 1 para tirar essa diferenç
          result.save("descriptografado.png")
          print("Descriptografia concluída!")
          break;
      im = im[xmap,ymap]

elif res == 3:
  c = int(input("Criptografar até qual periodo?\n"))
  for i in range(c+1):

    result = Image.fromarray(im)
    fin = result.resize(image.size)
    if (i  == c):
        fin.save("criptografado.png")
        print("Criptografia concluída!")
        break;
    im = im[xmap,ymap]
    i=i+1